current problem that should be discussed -
# todo : currently each regular is map to 1-5 different corp sentences, BUT it's still possible that 2 different regulars would be mapped to same corp sentences, same could be the opposite. for example - in data_sample_corp_to_1_regular each corp sentence is mapped to a single regular sentence, but since they don't have to be different, more than 2 different corp can be mapped into "No."


import packages:

In [100]:
import pandas as pd
from pathlib import Path

load data

In [107]:
dataset_path = 'initial_dataset/data_sample.csv'
data = pd.read_csv(dataset_path)
dataset_name = Path(dataset_path).stem

preprocess

In [108]:
def clean_col(df, col_name):
    df[col_name] = df[col_name].apply(lambda x: x.strip().capitalize())
    df[col_name] = df[col_name].apply(lambda x: x if x.endswith('.') else x + ".")
    return df

In [109]:
data = clean_col(data, "regular")
data = clean_col(data, "corp")
data = data.drop_duplicates()
data

,regular,corp
0,No.,"I appreciate the offer, but i can't."
3,No.,"I'm honored, but can't."
4,No.,"I'd love to, but i can't."
5,No.,"I appreciate the invitation, but i am complete..."
6,No.,"Thanks for thinking of me, but i can't."
7,No.,"Regrettably, i'm not able to."
8,No.,"You're so kind to think of me, but i can't."
9,No.,"No thank you, but it sounds lovely."
10,No.,"Thanks for the offer, but i can't."
11,No.,"Thanks, but maybe another time."


### save multi to multi version - (data is only cleaned)

In [110]:
data.to_csv('processed_dataset/multi_regular_to_multi_corp/' + dataset_name + '.csv')

"single - multi" ratios set (both for regular to corp and corp to regular)
for example -
multi_options = {1, 5}, so
1 -> 1:1 for each single will have 1 option
5 -> 1:5 for each single will have 5 options

In [132]:
multi_options = {1, 5}

# todo -  continue here, fix bug
def single_to_multi_explode_and_save(df, single_col, multi_col):
    single_to_multi_df = df.groupby(single_col)[multi_col].apply(set).apply(list).reset_index(name=multi_col)
    for i in multi_options:
        multi_col_i = f'{multi_col}_{i}'

        single_to_multi_df[multi_col_i] = single_to_multi_df[multi_col]\
            .apply(lambda lst: list(dict.fromkeys(lst))[:min(i, len(lst))])

        df_temp_i = single_to_multi_df.explode(multi_col_i)
        df_temp_i = df_temp_i[[single_col, multi_col_i]]

        directory = f"processed_dataset/multi_{multi_col}_to_single_{single_col}/{dataset_name}_{single_col}_to_{i}_{multi_col}.csv"
        df_temp_i.to_csv(directory)

    return single_to_multi_df



For each regular - pick first 5 corp sentences (no corp dup are allowed for each reg)

In [136]:
reg_to_multi_corp = single_to_multi_explode_and_save(data, 'regular', 'corp')
reg_to_multi_corp

,regular,corp,corp_1,corp_5
0,I don't have the time.,"[Perhaps next season when things clear up., Ci...",[Perhaps next season when things clear up.],"[Perhaps next season when things clear up., Ci..."
1,I told you so.,"[As per my prediction, this outcome does not c...","[As per my prediction, this outcome does not c...","[As per my prediction, this outcome does not c..."
2,No.,"[No thank you, but it sounds lovely., Regretta...","[No thank you, but it sounds lovely.]","[No thank you, but it sounds lovely., Regretta..."
3,That meeting sounds like a waste of my time.,[I'm unable to add value to this meeting but i...,[I'm unable to add value to this meeting but i...,[I'm unable to add value to this meeting but i...
4,That sounds like a horrible idea.,[Are we confident that this is the best soluti...,[Are we confident that this is the best soluti...,[Are we confident that this is the best soluti...
5,You are overcomplicating this.,[Being mindful of timelines. let?s concentrate...,[Being mindful of timelines. let?s concentrate...,[Being mindful of timelines. let?s concentrate...


In [137]:
multi_reg_to_corp = single_to_multi_explode_and_save(data, 'corp', 'regular')
multi_reg_to_corp

,corp,regular,regular_1,regular_5
0,Are we confident that this is the best solutio...,[That sounds like a horrible idea.],[That sounds like a horrible idea.],[That sounds like a horrible idea.]
1,"As per my prediction, this outcome does not co...",[I told you so.],[I told you so.],[I told you so.]
2,Ask me in a month.,[I don't have the time.],[I don't have the time.],[I don't have the time.]
3,"Bandwidth is low, so i won't be able to make i...",[I don't have the time.],[I don't have the time.],[I don't have the time.]
4,Being mindful of timelines. let?s concentrate ...,[You are overcomplicating this.],[You are overcomplicating this.],[You are overcomplicating this.]
5,Circle back to me in a few weeks.,[I don't have the time.],[I don't have the time.],[I don't have the time.]
6,"I appreciate the invitation, but i am complete...",[No.],[No.],[No.]
7,"I appreciate the offer, but i can't.",[No.],[No.],[No.]
8,I have a few things i need to take care of fir...,[I don't have the time.],[I don't have the time.],[I don't have the time.]
9,I wish i could make it work.,[No.],[No.],[No.]
